In [1]:
import os
import openai
import langchain
import streamlit as st
import pinecone
import langchain_pinecone
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
from langchain.chains.question_answering import load_qa_chain
from openai import OpenAI

c:\Users\tempj\anaconda3\envs\wiCodeAgentEnv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# Pinecone and OpenAI setup
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')

In [4]:
def initialize_components():
    # Initialize the Pinecone vector store
    pc_api_key = os.getenv('PINECONE_API_KEY')
    pc_index_name = os.getenv('PINECONE_INDEX_NAME')
    pc = Pinecone(api_key=pc_api_key)
    index = pc.Index(pc_index_name)
    
    # Initialize OpenAI embeddings and chat model
    api_key = os.getenv('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    # Creating the vector store with Pinecone and the embeddings
    vectorstore = PineconeVectorStore.from_existing_index(
        index_name=pc_index_name,
        embedding=embeddings
    )        
    
    return embeddings, vectorstore, api_key

In [16]:
def get_conversational_chain(api_key):
    template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n
    
    
    Answer:
    """
    chat_model = ChatOpenAI(openai_api_key=api_key, model_name='gpt-4-0125-preview', temperature=0.6)
    prompt = ChatPromptTemplate.from_template(template)
    chain = load_qa_chain(chat_model, chain_type="stuff", prompt=prompt)
    return chain

In [20]:
def user_input(user_question, api_key, vectorstore, embeddings):
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
    docs = retriever.invoke(user_question)
    chain = get_conversational_chain(api_key)
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
    return response


In [23]:
embeddings, vectorstore, api_key = initialize_components()
user_question = "What's the live load for garage floors?"

response = user_input(user_question, api_key, vectorstore, embeddings)

# Now print the response variable
print(response)

{'output_text': 'The live load for garage floors is 50 pounds per square foot.'}
